# 文本回归

In [1]:
import os

import numpy as np
import tensorflow as tf
from sklearn.datasets import load_files

import autokeras as ak

为了使本教程易于理解，我们仅将 IMDB 数据集视为回归数据集。 这意味着我们将 IMDB 数据集的预测目标，即 0 和 1 视为数值，以便它们可以直接用作回归目标。

In [2]:
gpus = tf.config.list_physical_devices("GPU")

if gpus:
    gpu0 = gpus[1] #如果有多个GPU，仅使用第0个GPU
    tf.config.experimental.set_memory_growth(gpu0, True) #设置GPU显存用量按需使用
    # 或者也可以设置GPU显存为固定使用量(例如：4G)
    #tf.config.experimental.set_virtual_device_configuration(gpu0,
    #    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)]) 
    tf.config.set_visible_devices([gpu0],"GPU") 

In [3]:
gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU')]

## 一个简单的例子

第一步是准备数据。 这里我们以 IMDB 数据集为例。

In [4]:
dataset = tf.keras.utils.get_file(
    fname="aclImdb.tar.gz",
    origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz",
    extract=True,
)

In [5]:
# set path to dataset
IMDB_DATADIR = os.path.join(os.path.dirname(dataset), "aclImdb")

In [6]:
classes = ["pos", "neg"]
train_data = load_files(
    os.path.join(IMDB_DATADIR, "train"), shuffle=True, categories=classes
)
test_data = load_files(
    os.path.join(IMDB_DATADIR, "test"), shuffle=False, categories=classes
)

In [7]:
x_train = np.array(train_data.data)
y_train = np.array(train_data.target)
x_test = np.array(test_data.data)
y_test = np.array(test_data.target)

print(x_train.shape)  # (25000,)
print(y_train.shape)  # (25000, 1)
print(x_train[0][:50])  # <START> this film was just brilliant casting <UNK>

(25000,)
(25000,)
b'Zero Day leads you to think, even re-think why two'


第二步是运行TextRegressor。 作为一个快速演示，我们将 epochs 设置为 2。您还可以为自适应数量的 epochs 保留未指定的 epochs。

In [9]:
# Initialize the text regressor.
reg = ak.TextRegressor(overwrite=True, max_trials=1)  # It tries 10 different models.

In [10]:
# Feed the text regressor with training data.
reg.fit(x_train, y_train, epochs=2)

Trial 1 Complete [00h 01m 02s]
val_loss: 0.15142059326171875

Best val_loss So Far: 0.15142059326171875
Total elapsed time: 00h 01m 02s
INFO:tensorflow:Oracle triggered exit
Epoch 1/2
782/782 [==============================] - 27s 34ms/step - loss: 0.1838 - mean_squared_error: 0.1838
Epoch 2/2
782/782 [==============================] - 26s 33ms/step - loss: 0.1230 - mean_squared_error: 0.1230
INFO:tensorflow:Assets written to: ./text_regressor/best_model/assets


In [11]:
# Predict with the best model.
predicted_y = reg.predict(x_test)

782/782 [==============================] - 16s 20ms/step


In [12]:
# Evaluate the best model with testing data.
print(reg.evaluate(x_test, y_test))

782/782 [==============================] - 20s 25ms/step - loss: 0.1560 - mean_squared_error: 0.1560
[0.1560177057981491, 0.1560177057981491]


## 验证数据

默认情况下，AutoKeras 使用最后 20% 的训练数据作为验证数据。 如下例所示，您可以使用validation_split 来指定百分比。

In [14]:
reg.fit(
    x_train,
    y_train,
    # Split the training data and use the last 15% as validation data.
    validation_split=0.15,
)

您还可以使用自己的验证集，而不是使用validation_data 将其从训练数据中分离出来。

In [15]:
split = 5000
x_val = x_train[split:]
y_val = y_train[split:]
x_train = x_train[:split]
y_train = y_train[:split]
reg.fit(
    x_train,
    y_train,
    epochs=2,
    # Use your own validation set.
    validation_data=(x_val, y_val),
)

## 自定义搜索空间

对于高级用户，您可以使用 AutoModel 而不是 TextRegressor 来自定义您的搜索空间。 您可以为一些高级配置配置 TextBlock，例如，针对要使用的文本矢量化方法类型的矢量化器。 您可以使用“sequence”，它使用 TextToInteSequence 将单词转换为整数并使用 Embedding 嵌入整数序列，或者您可以使用“ngram”，它使用 TextToNgramVector 对句子进行向量化。 您也可以不指定这些参数，这将使不同的选择自动调整。 有关详细信息，请参阅以下示例。

In [16]:
input_node = ak.TextInput()
output_node = ak.TextBlock(block_type="ngram")(input_node)
output_node = ak.RegressionHead()(output_node)
reg = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=1
)
reg.fit(x_train, y_train, epochs=2)

Trial 1 Complete [00h 00m 14s]
val_loss: 0.36922892928123474

Best val_loss So Far: 0.36922892928123474
Total elapsed time: 00h 00m 14s
INFO:tensorflow:Oracle triggered exit
Epoch 1/2
157/157 [==============================] - 5s 29ms/step - loss: 0.5417 - mean_squared_error: 0.5417
Epoch 2/2
157/157 [==============================] - 5s 29ms/step - loss: 0.4120 - mean_squared_error: 0.4120
INFO:tensorflow:Assets written to: ./auto_model/best_model/assets


AutoModel 的用法类似于 Keras 的函数式 API。 基本上，您正在构建一个图，其边是块，节点是块的中间输出。 使用 output_node = ak.[some_block]([block_args])(input_node) 添加从 input_node 到 output_node 的边。

您甚至还可以使用更细粒度的块来进一步自定义搜索空间。 请参阅以下示例。

In [17]:
input_node = ak.TextInput()
output_node = ak.TextToIntSequence()(input_node)
output_node = ak.Embedding()(output_node)
# Use separable Conv layers in Keras.
output_node = ak.ConvBlock(separable=True)(output_node)
output_node = ak.RegressionHead()(output_node)
reg = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=1
)
reg.fit(x_train, y_train, epochs=2)

Trial 1 Complete [00h 00m 15s]
val_loss: 0.19610105454921722

Best val_loss So Far: 0.19610105454921722
Total elapsed time: 00h 00m 15s
INFO:tensorflow:Oracle triggered exit
Epoch 1/2
157/157 [==============================] - 7s 41ms/step - loss: 0.2793 - mean_squared_error: 0.2793
Epoch 2/2
157/157 [==============================] - 6s 38ms/step - loss: 0.2067 - mean_squared_error: 0.2067
INFO:tensorflow:Assets written to: ./auto_model/best_model/assets


## 数据格式

AutoKeras TextRegressor 对于数据格式非常灵活。

对于文本，输入数据应该是一维的，对于回归目标，应该是数值向量。 AutoKeras 接受 numpy.ndarray。

我们还支持对训练数据使用 tf.data.Dataset 格式。

In [18]:
train_set = tf.data.Dataset.from_tensor_slices(((x_train,), (y_train,))).batch(32)
test_set = tf.data.Dataset.from_tensor_slices(((x_test,), (y_test,))).batch(32)

In [19]:
reg = ak.TextRegressor(overwrite=True, max_trials=2)
# Feed the tensorflow Dataset to the regressor.
reg.fit(train_set, epochs=2)

Trial 2 Complete [00h 00m 14s]
val_loss: 0.19924014806747437

Best val_loss So Far: 0.19136081635951996
Total elapsed time: 00h 00m 27s
INFO:tensorflow:Oracle triggered exit
Epoch 1/2
157/157 [==============================] - 6s 33ms/step - loss: 0.2826 - mean_squared_error: 0.2826
Epoch 2/2
157/157 [==============================] - 5s 33ms/step - loss: 0.2010 - mean_squared_error: 0.2010
INFO:tensorflow:Assets written to: ./text_regressor/best_model/assets


In [20]:
# Predict with the best model.
predicted_y = reg.predict(test_set)

782/782 [==============================] - 16s 20ms/step


In [21]:
# Evaluate the best model with testing data.
print(reg.evaluate(test_set))

782/782 [==============================] - 20s 26ms/step - loss: 0.1985 - mean_squared_error: 0.1985
[0.19846472144126892, 0.19846472144126892]
